In [ ]:
app = Flask(__name__)
@app.route('/predict/<string:sentence>', methods=['GET'])
def predict1(sentence):
    # if this language of sentence is arabic
    if detectText(sentence) =='ar':
        # Load first model of arabic which represent Subjecitve or Objective class
        model1 = torch.load('Arabic_Subjecitive_Objective.pt',map_location=torch.device('cpu'))
        # Load Tokenizer for MARBERT transformer
        tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")
        # Make Auto Correction to arabic sentence
        sentence_after_correction = corr.contextual_correct(sentence)
        temp = sentence_after_correction
        # Make  Arabic Cleaning process to sentence
        sentence_after_correction_cleaned = clean_text_arabic(sentence_after_correction)
        # Make predict (subjective - objective) (1 - 0)
        result = predict(sentence_after_correction_cleaned,model1 , tokenizer , 106)
        # Result is zero mean it Objective (Postive - Negative )
        if str(result) =='0':
            # Load second model which represent Positive or Negative
            model2 = torch.load('Arabic_Postive_Negative.pt',map_location=torch.device('cpu'))
            # Make Cleaning proecces to sentence
            sentence_after_correction_cleaned = clean_text_arabic(temp)
            # Make Predict (postive - negative)(1 - 0)
            result = predict(sentence_after_correction_cleaned,model2,tokenizer, 106)
            return str(result)
        # Otherwise mean it Subective (Neutral)
        else:
            return '-1'
    # this language of sentence is english
    else:
         # Load first model of english which represent Subjecitve or Objective class
        model1 = torch.load('English_subjective_objective.pt',map_location=torch.device('cpu'))
        # Load tokenizer for distilbert-base-uncased transformer
        tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
          # Make Auto Correction to english sentence
        sentence_after_correction = spell(sentence)
        temp = sentence_after_correction
        # Make Engish Cleaning process to sentence
        sentence_after_correction_cleaned = clean_text_english(sentence_after_correction)
        # Make predict (subjective - objective) (1 - 0)
        result = predict(sentence_after_correction_cleaned,model1 , tokenizer , 128)
         # Result is zero mean it Objective (Postive - Negative )
        if str(result) =='0':
            # Load second model which represent Positive or Negative
            model2 = torch.load('English_pos_neg.pt',map_location=torch.device('cpu'))
            # Make Engish Cleaning proecces to sentence
            sentence_after_correction_cleaned = clean_text_english(temp)
             # Make Predict (postive - negative)(1 - 0)
            result = predict(sentence_after_correction_cleaned,model2 , tokenizer , 128)
            return str(result)
        # Otherwise mean it Subective (Neutral)
        else:
            return '-1'
        

In [ ]:
@app.route('/summaryText/<string:text>', methods=['GET'])
def summary_text(text):
    # Represent name of summary model
    model_name = "pszemraj/led-large-book-summary"
    # Load Toknizer for summarization
    tokenizer = LEDTokenizer.from_pretrained(model_name)
    # The model is also initialized with the pre-trained model name defined above
    model = LEDForConditionalGeneration.from_pretrained(model_name)
    # The encoder method 
    inputs = tokenizer.encode(text, padding="max_length", truncation=True, max_length=4096, return_tensors="pt")
    # Summary of encoded text using pretrained model
    summary_ids = model.generate(inputs, max_length=512, num_beams=5, early_stopping=True)
    # Decode the summary output using toknizer 
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
@app.route('/summaryYoutube/<string:video_id>', methods=['GET'])
def summary_youtube(video_id):
    # Feach content of video with it's id
    textParts = YouTubeTranscriptApi.get_transcript(video_id)
    # Merge parts of texts togather
    text = ' '.join([c['text'] for c in textParts])
    # Represent name of summary model
    model_name = "pszemraj/led-large-book-summary"
    # Load Toknizer for summarization
    tokenizer = LEDTokenizer.from_pretrained(model_name)
    # The model is also initialized with the pre-trained model name defined above
    model = LEDForConditionalGeneration.from_pretrained(model_name)
    # The encoder method 
    inputs = tokenizer.encode(text, padding="max_length", truncation=True, max_length=4096, return_tensors="pt")
    # Summary of encoded text using pretrained model
    summary_ids = model.generate(inputs, max_length=512, num_beams=5, early_stopping=True)
    # Decode the summary output using toknizer 
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary